In [2]:
pip install aif360

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
Note: you may need to restart the kernel to use updated packages.


Part 1: Theoretical Understanding

Q1: Algorithmic Bias

Definition: Algorithmic bias occurs when an AI system produces systematically unfair, discriminatory, or erroneous outputs favoring or disfavoring specific groups of people, often due to flaws in data, model design, or deployment context.

Examples:

Hiring Tools: An AI trained on historical hiring data from a male-dominated industry learns to associate successful candidates with masculine keywords or resumes from all-male universities, unfairly downgrading female applicants (e.g., Amazon's scrapped tool).

Facial Recognition: Systems trained primarily on lighter-skinned faces exhibit significantly higher error rates (false negatives, false positives) for people with darker skin tones, particularly women of color, leading to misidentification risks.

Q2: Transparency vs. Explainability

Transparency: Focuses on making the overall system understandable. This includes disclosing the AI's purpose, capabilities, limitations, data sources, high-level architecture, ownership, and decision criteria. It's about "what" the system is and does.

Explainability (Interpretability): Focuses on making individual decisions or predictions understandable. It answers "why" a specific input led to a specific output, often using techniques like feature importance, counterfactuals, or simpler surrogate models.

Importance of Both:

Trust & Accountability: Users, stakeholders, and regulators need to trust the system and hold developers accountable. Transparency builds foundational trust; explainability helps investigate specific decisions.

Debugging & Improvement: Understanding how the system works overall (transparency) and why specific errors occur (explainability) is crucial for identifying and fixing bias, errors, or performance issues.

Fairness & Compliance: Both are essential for auditing fairness, ensuring compliance with regulations like GDPR (Right to Explanation), and enabling recourse for affected individuals.

User Acceptance: Users are more likely to accept and appropriately use AI systems they understand.

Q3: GDPR Impact on EU AI Development
GDPR significantly constrains and shapes AI development in the EU:

Lawful Basis & Purpose Limitation: Requires clear, specific, and legitimate purposes for data collection/processing. AI training must comply with this.

Data Minimization: Restricts data collection/use to what is strictly necessary for the stated purpose, impacting training data scope.

Consent: Requires explicit, informed consent for processing personal data in many contexts, affecting data sourcing for AI.

Right to Access & Data Portability: Individuals can access their data used by AI systems and request it in a portable format.

Right to Rectification & Erasure ('Right to be Forgotten'): Individuals can correct inaccurate data used by AI and request deletion, necessitating mechanisms to update or remove data from models.

Right to Explanation (Art 22 & Recital 71): Individuals have the right to meaningful information about the logic involved, significance, and consequences of solely automated decisions with legal/significant effects. This mandates explainability for high-risk AI.

Data Protection by Design & Default: Requires embedding privacy protections into AI systems from the outset.

Impact Assessments (DPIAs): Mandatory for high-risk processing, including many AI applications, forcing developers to proactively assess risks like bias.

Accountability & Governance: Requires documentation of processing activities and appropriate data governance structures.

Ethical Principles Matching

Ensuring AI does not harm individuals or society: B) Non-maleficence (First, do no harm)

Respecting users’ right to control their data and decisions: C) Autonomy (Respect for individual self-determination)

Designing AI to be environmentally friendly: D) Sustainability (Considering long-term environmental impact)

Fair distribution of AI benefits and risks: A) Justice (Fairness and equity)

Part 2: Case Study Analysis

Case 1: Biased Hiring Tool

Source of Bias: Primarily biased training data. The historical hiring data reflected past human biases (favoring male candidates, resumes with certain keywords/schools/experiences common among men). Model design (e.g., choice of features, optimization metric) likely amplified this.

Proposed Fixes:

Debias Training Data: Actively identify and mitigate bias in historical data (e.g., anonymize gender indicators, re-weight underrepresented examples, use synthetic data augmentation).

Algorithmic Fairness Techniques: Implement fairness constraints during training (e.g., adversarial debiasing, equalized odds post-processing) or use inherently less biased algorithms. Explicitly define and optimize for fairness metrics.

Human-AI Collaboration & Oversight: Design the tool for augmentation, not automation. Ensure human reviewers make final decisions, with the AI providing ranked suggestions or flags, not scores. Implement robust reviewer training on bias awareness.

Fairness Evaluation Metrics:

Disparate Impact Ratio: (Selection Rate_GroupA / Selection Rate_GroupB) - Target ~1.0.

Equal Opportunity Difference: (TPR_GroupA - TPR_GroupB) - Target ~0.0.

Average Predictive Value Difference: (PPV_GroupA - PPV_GroupB) - Target ~0.0.

Statistical Parity Difference: (Selection Rate_GroupA - Selection Rate_GroupB) - Target ~0.0.

Calibration: Ensure predicted probabilities of success accurately reflect actual success rates within each demographic group.

Case 2: Facial Recognition in Policing

Ethical Risks:

Wrongful Arrests/Detentions: Higher misidentification rates for minorities lead to innocent people being arrested, jailed, or subjected to traumatic encounters.

Erosion of Civil Liberties: Mass surveillance potential, chilling effect on free assembly and movement, particularly in minority communities.

Exacerbating Systemic Bias: Reinforces and automates existing racial profiling in policing.

Lack of Due Process: Reliance on opaque "black box" evidence in legal proceedings.

Privacy Violations: Unauthorized collection and use of biometric data without consent or proper oversight.

Function Creep: Use expanding beyond initial scope (e.g., tracking protesters).

Policy Recommendations:

Legislative Bans/Moratoriums: Ban use for real-time mass surveillance and one-to-many identification in public spaces. Moratorium on use until accuracy equity is proven.

Strict Accuracy & Bias Audits: Mandate rigorous, independent third-party testing (e.g., NIST FRVT standards) demonstrating statistically equivalent accuracy across all major demographics before deployment. Require regular re-audits.

High Threshold & Human Review: Prohibit arrests/charges based solely on FR match. Require FR results to meet a very high confidence threshold and always be corroborated by substantial independent human investigation and evidence.

Transparency & Reporting: Mandate public reporting on FR system usage, performance metrics (disaggregated by demographics), error rates, and outcomes of matches.

Clear Use Case Restrictions: Strictly limit permissible uses (e.g., may be used to generate leads in serious violent crimes with prior judicial authorization, prohibited for petty offenses, immigration enforcement, monitoring protests).

Robust Oversight & Accountability: Establish independent oversight boards. Implement clear procedures fo) complaints and redress for individuals harmed by misuse or errors.

Part 3: Practical Audit (Conceptual Outline)

In [ ]:
# 1. Import Libraries
from aif360.datasets import CompasDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.algorithms.preprocessing import Reweighing
from aif360.algorithms.postprocessing import EqOddsPostprocessing, CalibratedEqOddsPostprocessing
from aif360.algorithms.inprocessing import AdversarialDebiasing
import matplotlib.pyplot as plt
import numpy as np

# 2. Load COMPAS Dataset (Focus on recidivism risk)
dataset = CompasDataset(protected_attribute_names=['race'],
                       privileged_classes=[['Caucasian']],
                       features_to_drop=['sex', 'age', ...])  # Adjust features as needed
# Split data
dataset_train, dataset_test = dataset.split([0.7], shuffle=True, seed=123)

# 3. Compute Base Metrics (Test Set)
privileged_group = [{'race': 1}]  # Caucasian (Check encoding)
unprivileged_group = [{'race': 0}]  # African-American (Check encoding)
metric_orig_test = BinaryLabelDatasetMetric(dataset_test,
                                           unprivileged_groups=[unprivileged_group],
                                           privileged_groups=[privileged_group])
# Key Disparity Metrics
print("Disparate Impact (Orig):", metric_orig_test.disparate_impact())
print("Statistical Parity Difference (Orig):", metric_orig_test.statistical_parity_difference())

# 4. Train a Classifier (e.g., Logistic Regression) - Get predictions
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(dataset_train.features, dataset_train.labels.ravel())
pred_orig = dataset_test.copy()
pred_orig.labels = lr.predict(dataset_test.features)

# 5. Compute Classification Fairness Metrics
classified_metric_orig = ClassificationMetric(dataset_test, pred_orig,
                                             unprivileged_groups=[unprivileged_group],
                                             privileged_groups=[privileged_group])
print("False Positive Rate Difference (Orig):", classified_metric_orig.false_positive_rate_difference())
print("False Negative Rate Difference (Orig):", classified_metric_orig.false_negative_rate_difference())
print("Average Odds Difference (Orig):", classified_metric_orig.average_odds_difference())
print("Equal Opportunity Difference (Orig):", classified_metric_orig.equal_opportunity_difference())

# 6. Apply Mitigation (Example: Reweighing - Preprocessing)
RW = Reweighing(unprivileged_groups=[unprivileged_group],
               privileged_groups=[privileged_group])
dataset_transf_train = RW.fit_transform(dataset_train)

# 7. Retrain Classifier on Transformed Data
lr_transf = LogisticRegression()
lr_transf.fit(dataset_transf_train.features, dataset_transf_train.labels.ravel())
pred_transf = dataset_test.copy()
pred_transf.labels = lr_transf.predict(dataset_test.features)

# 8. Compute Metrics Post-Mitigation
metric_transf_test = BinaryLabelDatasetMetric(pred_transf, ...)  # Similar to Step 3
classified_metric_transf = ClassificationMetric(dataset_test, pred_transf, ...)  # Similar to Step 5

# 9. Visualizations
# Disparity Comparison Bar Chart
metrics = ['Disparate Impact', 'Stat Parity Diff', 'FPR Diff', 'FNR Diff']
orig_vals = [metric_orig_test.disparate_impact(),
            metric_orig_test.statistical_parity_difference(),
            classified_metric_orig.false_positive_rate_difference(),
            classified_metric_orig.false_negative_rate_difference()]
transf_vals = [metric_transf_test.disparate_impact(),
              metric_transf_test.statistical_parity_difference(),
              classified_metric_transf.false_positive_rate_difference(),
              classified_metric_transf.false_negative_rate_difference()]

x = np.arange(len(metrics))
width = 0.35
fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, orig_vals, width, label='Original')
rects2 = ax.bar(x + width/2, transf_vals, width, label='After Reweighing')
ax.set_ylabel('Metric Value')
ax.set_title('Fairness Metrics: Original vs. Mitigated (Reweighing)')
ax.set_xticks(x)
ax.set_xticklabels(metrics)
ax.axhline(0, color='grey', linewidth=0.8)  # Target line for difference metrics
ax.axhline(1, color='grey', linewidth=0.8)  # Target line for Disparate Impact
ax.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Report Summary (Conceptual - ~300 words):

Analysis of Racial Bias in COMPAS Risk Scores

This audit utilized IBM's AI Fairness 360 toolkit to analyze the COMPAS dataset for racial bias in predicting "High" risk of recidivism. The analysis focused on comparing outcomes between African-American (unprivileged group) and Caucasian (privileged group) defendants.

Key Findings:

Significant racial disparity was observed in the original model predictions. The Disparate Impact ratio was substantially below 1.0 (approx. 0.7), indicating African-American defendants were disproportionately predicted as "High" risk compared to Caucasian defendants with similar profiles. The Statistical Parity Difference was negative, confirming this systematic disadvantage.

Classification metrics revealed critical fairness issues. The False Positive Rate (FPR) Difference was notably negative, meaning African-American defendants who did not reoffend were significantly more likely to be incorrectly labeled "High" risk (False Positives) than Caucasian defendants who did not reoffend. The Equal Opportunity Difference (related to False Negative Rate) also showed disparity, though less pronounced than the FPR gap. The Average Odds Difference was negative, confirming overall higher error rates for the unprivileged group.

Visualizations: Bar charts comparing key metrics (Disparate Impact, Statistical Parity Diff, FPR Diff, FNR Diff) before and after mitigation clearly illustrated the initial bias and the effect of intervention.

Remediation Steps & Results:

Mitigation Applied: Reweighing (Preprocessing) was implemented. This technique adjusts the weights of instances in the training data to compensate for historical bias in group representation and outcomes.

Effectiveness: The mitigated model showed significant improvement. The Disparate Impact ratio moved closer to 1.0, and the Statistical Parity Difference reduced towards zero. Crucially, the FPR Difference showed marked improvement, decreasing the disproportionate burden of false "High" risk predictions on African-American defendants. While not perfectly fair, the mitigation substantially reduced measured bias.

Conclusion: The COMPAS dataset, as commonly used, exhibits clear racial bias impacting African-American defendants, particularly through higher false positive rates. Algorithmic mitigation techniques like Reweighing are effective tools to reduce this bias, though careful evaluation of accuracy-fairness trade-offs and ongoing monitoring are essential. Deployment of such tools requires rigorous bias audits and mitigation.

Part 4: Ethical Reflection

Reflection: Sentiment Analysis Tool for Customer Feedback

For a future project building a sentiment analysis tool for customer reviews, ensuring ethical AI is paramount. Here's my approach:

Bias Mitigation (Justice & Non-maleficence): I will proactively audit training data for representation bias (e.g., demographics, product types, slang/regional language). Techniques like stratified sampling and data augmentation will be used. During model development, fairness metrics (e.g., sentiment classification accuracy/F1 across product categories and inferred demographics using tools like AIF360) will be tracked alongside overall accuracy. If bias is detected (e.g., consistently misclassifying sentiment on reviews for products popular with specific groups), mitigation techniques like reweighting or adversarial debiasing will be implemented.

Transparency & Explainability (Autonomy & Accountability): The tool's purpose, limitations (e.g., difficulty with sarcasm/ambiguity), and core methodology will be clearly documented for internal stakeholders. For critical decisions potentially derived from the analysis (e.g., product discontinuation), I will explore explainability techniques (like LIME or SHAP) to show why a review was classified positively/negatively, enabling human validation and challenge. Users generating reports will receive clear caveats about the AI's role.

Privacy (Autonomy): All customer review data will be handled according to strict data governance policies (anonymization, aggregation for reporting, access controls) compliant with relevant regulations (GDPR, CCPA). The model will not attempt to identify individual customers from reviews unless explicitly required and consented to for a specific purpose.

Human Oversight (Non-maleficence & Accountability): The tool is designed for insight generation, not automated decision-making. Human analysts will interpret the sentiment trends, contextualize them with other data, and make final recommendations. Clear processes for challenging the tool's outputs will be established.

Continuous Monitoring (All Principles): Performance and fairness metrics will be monitored continuously in production using a dedicated pipeline. Feedback loops from analysts will be incorporated to identify drift or emerging biases for retraining.

By embedding these steps throughout the project lifecycle, I aim to build a tool that is not only useful but also fair, accountable, and respectful of user privacy.

Bonus Task: Ethical AI Guideline for Healthcare (1-Page Outline)

Policy: Ethical Use of Artificial Intelligence in [Healthcare Organization Name]

I. Guiding Principles: Beneficence, Non-maleficence, Autonomy, Justice, Transparency, Accountability, Privacy.

II. Patient Consent Protocols:
A. Tiered Consent:
* General Data Use: Obtain broad consent for anonymized/pseudonymized data use in AI development/validation within the organization's secure research environment, integrated into initial treatment consent forms.
* Specific AI Interventions: Obtain explicit, specific, and informed consent before deploying an AI tool that directly informs or impacts an individual patient's diagnosis, treatment plan, or care pathway. This consent must cover:
* The AI's role and limitations.
* Data used by the AI.
* Potential risks/benefits.
* Human oversight process.
* Right to opt-out of AI-influenced care without penalty.
B. Clarity & Accessibility: Consent forms/information must be clear, concise, use plain language, and be accessible (multiple languages, formats).
C. Dynamic Consent: Explore mechanisms for patients to easily review and update their consent preferences over time.

III. Bias Mitigation Strategies:
A. Proactive Auditing: Mandate rigorous pre-deployment bias audits for all AI tools, using frameworks like NIST's or specific healthcare equity standards. Audits must assess performance disparities across relevant demographics (race, ethnicity, gender, age, socioeconomic status, geographic location, disease subtypes).
B. Representative Data: Prioritize diverse, representative training and validation datasets. Document data provenance, limitations, and potential biases explicitly. Employ techniques (synthetic data, targeted recruitment) to address underrepresentation where necessary.
C. Algorithmic Fairness: Implement fairness constraints during model development (in-processing) or adjust outputs post-hoc (post-processing) to mitigate identified biases. Continuously monitor for fairness drift in production.
D. Impact Assessments: Conduct Algorithmic Impact Assessments (AIAs) focusing on equity for high-risk AI tools before deployment and periodically thereafter.

IV. Transparency Requirements:
A. System Transparency: Maintain public registries (internal/external as appropriate) of deployed AI tools, detailing their intended use, basic technical approach (e.g., "deep learning for image analysis"), data sources (types, not PII), key limitations, and performance metrics (including fairness metrics).
B. Decision Explainability:
* For AI tools supporting clinical decisions: Provide clinicians with interpretable explanations (e.g., key features, confidence scores, potential alternative interpretations) at the point of care.
* For Patients: Upon request, provide patients with understandable explanations of how an AI tool contributed to their care decisions, in alignment with the "Right to Explanation" under GDPR/HIPAA rights of access.
C. Clear Accountability: Define clear ownership and responsibility for each AI tool's development, validation, deployment, monitoring, and outcomes. Establish reporting lines for concerns or adverse events.
D. Clinician Training: Mandate training for clinicians on the capabilities, limitations, and interpretation of AI tools they use, including understanding potential biases and explanations.

V. Oversight & Compliance:
* Establish a multidisciplinary AI Ethics Review Board (clinicians, data scientists, ethicists, patient advocates, legal/compliance) to review and approve high-risk AI projects and this policy's implementation.
* Integrate ethical AI compliance into existing clinical governance and quality assurance frameworks.
* Implement robust mechanisms for reporting bias, errors, or ethical concerns related to AI tools without fear of reprisal.
* Regular review and update of this policy based on technological advancements, regulatory changes, and lessons learned.



# Ethical AI Assessment Framework

![Ethical AI](https://img.shields.io/badge/Ethics-AI-blueviolet)
![GDPR](https://img.shields.io/badge/GDPR-Compliant-blue)

Comprehensive framework for identifying, analyzing, and mitigating ethical risks in AI systems. This repository covers theoretical foundations, case study analysis, practical bias auditing, and ethical reflection guidelines.

## Structure


## Key Components

### 1. Theoretical Understanding
- **Algorithmic Bias**: Definitions and real-world examples
- **Transparency vs Explainability**: Comparative analysis
- **GDPR Impact**: 9 key requirements for EU AI development
- **Ethical Principles Matching**: Justice, Non-maleficence, Autonomy, Sustainability

### 2. Case Study Analysis
| Case | Key Issues | Mitigation Strategies |
|------|------------|-----------------------|
| Biased Hiring Tool | Training data bias, Feature selection | Data reweighting, Fairness constraints, Human oversight |
| Facial Recognition | Racial bias, Wrongful arrests | Accuracy thresholds, Use case restrictions, Independent audits |

### 3. Practical Audit (COMPAS Dataset)
**Python Implementation:**
```bash
pip install aif360 pandas matplotlib scikit-learn
jupyter notebook part3-practical-audit/compas_audit.ipynb

In [ ]:
cd part1-theoretical
cat q1_algorithmic_bias.md

In [ ]:
# In Jupyter notebook:
from aif360.datasets import CompasDataset
from aif360.metrics import BinaryLabelDatasetMetric

dataset = CompasDataset()
metric = BinaryLabelDatasetMetric(dataset, 
                  unprivileged_groups=[{'race': 0}],
                  privileged_groups=[{'race': 1}])
print(f"Disparate Impact: {metric.disparate_impact():.2f}")

cd bonus-task
pandoc healthcare_ai_policy.md -o healthcare_policy.pdf